<a href="https://colab.research.google.com/github/PriyaMulla/GENNDER/blob/main/PRAW_reddit_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reddit- Auth Key (direct)
https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example

In [ ]:
from google.colab import userdata
rtpassword = userdata.get('rtpassword')
CLIENT_ID= userdata.get('client_id')
SECRET_KEY= userdata.get('secret_key')


#Reddit- Auth Key (direct) https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example

import requests

auth = requests.auth.HTTPBasicAuth('CLIENT_ID', 'SECRET_TOKEN')

data = {'grant_type': 'password',
        'username': 'Fitzkeys',
        'password': rtpassword}

headers = {'User-Agent': 'gender_bot_thesis/0.1 by u/Fitzkeys'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
res.json()

{'message': 'Unauthorized', 'error': 401}

## PRAW (wrapper)
https://asyncpraw.readthedocs.io/en/stable/getting_started/authentication.html#password-flow

In [ ]:
!pip install asyncpraw
import asyncpraw

#authorized reddit instance

reddit = asyncpraw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_KEY,
    password = rtpassword,
    user_agent='gender_bot_thesis/0.1 by u/fitzkeys',
    username = 'Fitzkeys'
)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 1.1 MB/s eta 0:00:00


## save reddit data

In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#input subreddit to get data from
sub_name = 'MensRights'

In [ ]:
directory = os.path.join('/content/drive/My Drive/Colab Notebooks/Thesis', sub_name)
if not os.path.exists(directory):
  os.mkdir(directory)

In [ ]:
#save data
def save_data(filename: str, reddit_data):
  reddit_data.to_csv(os.path.join('/content/drive/My Drive/Colab Notebooks/Thesis', sub_name, filename), index=False)

#get data
def get_data(filename:str):
  return pd.read_csv(os.path.join('/content/drive/My Drive/Colab Notebooks/Thesis', sub_name, filename), na_filter=False)

In [ ]:
#get posts
async def getPosts(subreddit_name: str, post_limit: int):
  subreddit = await reddit.subreddit(subreddit_name, fetch=True)
  subreddit_top =[]

  async for sub in subreddit.top(time_filter='all',limit=post_limit):
    if not(sub.selftext == '' and sub.title =='') and sub.num_comments>0:
      subreddit_top.append({
          'created': sub.created_utc,
          'link_flair_text': sub.link_flair_text,
          'post_id': sub,
          'score': sub.score,
          'subreddit': sub.subreddit,
          'text': sub.selftext,
          'title': sub.title,
          'upvote_ration': sub.upvote_ratio,
          'url': sub.url
          })

  return pd.DataFrame(subreddit_top)


#get comments
async def get_comments(posts):

  comments_top = []
  progress = 0
  posts_n = len(posts.index)

  for id in posts['post_id']:
    post = await reddit.submission(id, fetch=False)
    post.comment_sort = "top"
    await post.load()
    await post.comments.replace_more(limit=0) #top comments without replies

    for comment in post.comments.list()[:100]: #limit to top 100 comments
      if comment.body != '[deleted]' and comment.body != '[removed]': #dont save deleted or removed comments
        comments_top.append({'post_id': id, 'comment_id': comment.id, 'comment': comment.body})

    #print how many posts done
    progress = progress +1
    print("post ", progress, "/", posts_n, " done")
  return pd.DataFrame(comments_top)



#get comments and replies
async def get_comments_replies(posts):

  comments_replies_top = []
  progress = 0
  posts_n = len(posts.index)

  for id in posts['post_id']:
    post = await reddit.submission(id) #fetch=False
    comments = await post.comments()
    comments.reply_sort = 'top'
    await comments.replace_more(limit=0) #LIMIT RANGE FROM 0 TO 32/NONE (depends on how long it's taking)
    for comment in comments.list():
      if comment.body != '[deleted]' and comment.body != '[removed]': #dont save deleted or removed comments
        for reply in comment.replies:
          if reply.body != '[deleted]' and reply.body != '[removed]': #dont save deleted or removed replies
            comments_replies_top.append({'post_id': id, 'comment_id': comment.id, 'prompt': comment.body, 'response': reply.body})
            break

    #print how many posts done
    progress = progress +1
    print("post ", progress, "/", posts_n, " done")
    print("number of pairs: ", len(comments_replies_top))

  return pd.DataFrame(comments_replies_top)


### get posts

In [ ]:
posts =await getPosts(sub_name, 1000)
posts

,created,link_flair_text,post_id,score,subreddit,text,title,upvote_ration,url
0,1.625913e+09,[Discussion],ohgbq6,7499,Feminism,,World day without hijab,0.97,https://v.redd.it/f2gat7u35da71
1,1.610535e+09,[Discussion],kwe0oe,7284,Feminism,,Word.,0.97,https://i.redd.it/kvzb3y4vw2b61.jpg
2,1.618841e+09,[Discussion],mu1a9y,6923,Feminism,,It's called consent.,0.96,https://i.redd.it/9fmw4o9yz4u61.png
3,1.632227e+09,None,pshi2r,5934,Feminism,,Our bodies are not the problem!,0.96,https://i.redd.it/5594djrvluo71.jpg
4,1.549531e+09,[Abortion rights],ao1xoz,5772,Feminism,,Religious bigots are just monsters.,0.97,https://i.redd.it/5abqqj1454f21.jpg
...,...,...,...,...,...,...,...,...,...
993,1.604413e+09,[Quotes/Posters],jnb44g,1249,Feminism,,Vote ✊,0.96,https://i.redd.it/u8qvjktbb1x51.png
994,1.533067e+09,None,93huzx,1244,Feminism,,Feminism is for everybody,0.86,https://i.redd.it/wc5mk6mgacd11.jpg
995,1.706380e+09,None,1ach9xh,1237,Feminism,,The comments make me roll my eyes back into my...,0.97,https://i.redd.it/6sekbauh9uec1.jpeg
996,1.630955e+09,[Satire/Humor],pj5vq2,1241,Feminism,,Dang right I didn't!,0.98,https://i.redd.it/yxh6l8vkjxl71.jpg


In [ ]:
#save posts (raw data)
save_data('raw_posts.csv', posts)

### get comments
(for post_comment pairs, if getting replies don't run this)

In [ ]:
# usually takes around 10-30 min
comments= await get_comments(posts)
comments

In [ ]:
#save comments (raw data)
save_data('raw_comments.csv', comments)

### get comments and replies
(for comment_reply pairs, just needs posts for this)

In [ ]:
#took 14-25 min
posts = get_data('raw_posts.csv')
comments_replies= await get_comments_replies(posts)
comments_replies

In [ ]:
#save comments and replies (raw data)
save_data('raw_comments_replies.csv', comments_replies)

## [comment,reply]

In [ ]:
#get comments and replies
comments_replies = get_data('raw_comments_replies.csv').replace('', None).dropna().reset_index(drop=True)

In [ ]:
# add periods for parsing later
for i in range(len(comments_replies)):
  if comments_replies['prompt'][i][-1] not in ['.','?','!']:
    comments_replies['prompt'][i] = comments_replies['prompt'][i] + '.'
  if comments_replies['response'][i][-1] not in ['.','?','!']:
    comments_replies['response'][i] = comments_replies['response'][i] + '.'

In [ ]:
comments_replies

,post_id,comment_id,prompt,response
0,5o4pki,dcgqoo9,I was told to quit mansplaining on Friday by o...,Complain to HR about sexism. .
1,5o4pki,dcgpnyg,I've seen women do this more often than men on...,"Yes, but women sit like that because it's more..."
2,5o4pki,dcgnv82,"""I've never seen a well-mannered man on a plan...","The ""enjoy"" part was worse. Like some dumb bit..."
3,5o4pki,dcgo8i7,"""I've never seen a well-mannered man on a plan...",Something tells me these people aren't sardines .
4,5o4pki,dcgpxsb,I have to know how she responded to that comment.,Chances are she ignored it and acted like she ...
...,...,...,...,...
112834,zosw2k,j0pz7ic,So first you say it's because you commented \n...,I was using quotations to indicate speech in g...
112835,zosw2k,j2x38x7,"Maybe, but I still wish I'd been given the cho...","I guess that’s fair. But as of now, like u sai..."
112836,zosw2k,j0qwv8c,And another thing: even if it *did* reduce sen...,> If losing a little sensitivity allows me to ...
112837,zosw2k,j0qvy6k,I'm circumcised. I'm plenty sensitive. I have ...,>\tI’m circumcised. I’m plenty sensitive. I ha...


In [ ]:
#save final version
save_data('comment_reply.csv', comments_replies)

## [title+text, comment]


In [ ]:
#get posts and comments
posts = get_data('raw_posts.csv')
comments = get_data('raw_comments.csv')

In [ ]:
#merge comments df with posts df
combo = posts.merge(comments, on='post_id', how='outer')

#merge title and text
combo['post'] = combo['title'].astype(str) + '. ' + combo['text'] + '.'

#make sure comments have punctuation (for parsing later)
combo['comment'] = combo['comment'].astype(str) + '. '

final_data=combo.filter(['post_id', 'post', 'comment'])

final_data.rename(columns={'post_id': 'id', 'post': 'prompt', 'comment': 'response'}, inplace=True) #standardize column names
final_data

,id,prompt,response
0,5o4pki,The ignorance and loathing is real. .,I was told to quit mansplaining on Friday by o...
1,5o4pki,The ignorance and loathing is real. .,"""I've never seen a well-mannered man on a plan..."
2,5o4pki,The ignorance and loathing is real. .,"""I've never seen a well-mannered man on a plan..."
3,5o4pki,The ignorance and loathing is real. .,I've seen women do this more often than men on...
4,5o4pki,The ignorance and loathing is real. .,It's not always about you....
...,...,...,...
93031,zosw2k,Woman protesting infant circumcision in Ocean ...,Australians? Don’t throw us into the same boat...
93032,zosw2k,Woman protesting infant circumcision in Ocean ...,Consent is the issue. It can be the person who...
93033,zosw2k,Woman protesting infant circumcision in Ocean ...,I’m a male who wishes they weren’t. I’m 37. It...
93034,zosw2k,Woman protesting infant circumcision in Ocean ...,How many have you asked? I wish I'd been given...


In [ ]:
#save final version
save_data('title+text_comment.csv', final_data)

##[title, comment]

In [ ]:
#get posts and comments
posts = get_data('raw_posts.csv')
comments = get_data('raw_comments.csv')

In [ ]:
#merge comments df with posts df
comments_posts = posts.merge(comments, on='post_id', how='outer')

#make sure post and comments have punctuation (for parsing later)
comments_posts['title'] = comments_posts['title'].astype(str) + '.'
comments_posts['comment'] = comments_posts['comment'].astype(str) + '.'


final_data=comments_posts.filter(['post_id', 'title', 'comment'])


,post_id,title,comment
0,5o4pki,The ignorance and loathing is real.,I was told to quit mansplaining on Friday by o...
1,5o4pki,The ignorance and loathing is real.,"""I've never seen a well-mannered man on a plan..."
2,5o4pki,The ignorance and loathing is real.,"""I've never seen a well-mannered man on a plan..."
3,5o4pki,The ignorance and loathing is real.,I've seen women do this more often than men on...
4,5o4pki,The ignorance and loathing is real.,It's not always about you....
...,...,...,...
93031,zosw2k,Woman protesting infant circumcision in Ocean ...,Australians? Don’t throw us into the same boat...
93032,zosw2k,Woman protesting infant circumcision in Ocean ...,Consent is the issue. It can be the person who...
93033,zosw2k,Woman protesting infant circumcision in Ocean ...,I’m a male who wishes they weren’t. I’m 37. It...
93034,zosw2k,Woman protesting infant circumcision in Ocean ...,How many have you asked? I wish I'd been given...


In [ ]:
final_data.rename(columns={'post_id': 'id', 'post': 'prompt', 'comment': 'response'}, inplace=True)
final_data

,id,title,response
0,5o4pki,The ignorance and loathing is real.,I was told to quit mansplaining on Friday by o...
1,5o4pki,The ignorance and loathing is real.,"""I've never seen a well-mannered man on a plan..."
2,5o4pki,The ignorance and loathing is real.,"""I've never seen a well-mannered man on a plan..."
3,5o4pki,The ignorance and loathing is real.,I've seen women do this more often than men on...
4,5o4pki,The ignorance and loathing is real.,It's not always about you....
...,...,...,...
93031,zosw2k,Woman protesting infant circumcision in Ocean ...,Australians? Don’t throw us into the same boat...
93032,zosw2k,Woman protesting infant circumcision in Ocean ...,Consent is the issue. It can be the person who...
93033,zosw2k,Woman protesting infant circumcision in Ocean ...,I’m a male who wishes they weren’t. I’m 37. It...
93034,zosw2k,Woman protesting infant circumcision in Ocean ...,How many have you asked? I wish I'd been given...


In [ ]:
#save final version
save_data('title_comment.csv', final_data)

##[title+text, all_comments]

In [ ]:
#merge comments and posts
posts = get_data('raw_posts.csv')
comments = get_data('raw_comments.csv')

#merge comments df with posts df
comments_posts = posts.merge(comments, on='post_id', how='left')

#merge all comments together
combo = comments_posts[['post_id', 'title', 'text', 'comment']].astype(str)
combo = combo.groupby(['post_id', 'title', 'text'], sort=False, dropna=False)['comment'].apply('. '.join).reset_index()

#merge title and text
combo['post'] = combo['title'].astype(str) + '. ' + combo['text'] + '.'
combo['comment'] = combo['comment'].astype(str) + '.' #end with period

#keep just id, post, and comments
final_data = combo.filter(['post_id', 'post', 'comment'])
final_data.rename(columns={'post_id': 'id', 'post': 'prompt', 'comment': 'response'}, inplace=True)
final_data

,id,prompt,response
0,5o4pki,The ignorance and loathing is real. .,I was told to quit mansplaining on Friday by o...
1,60i0bi,Apparently Homelessness is only a Problem if y...,Homeless person here. I have never seen a home...
2,5j16nd,How to get banned from r/Feminism. .,It looks like they are behind on banning peopl...
3,6d1h12,Judge Judy Gets It. .,She's advocated for father's rights before too...
4,jprmy5,Petition to have Amber Heard removed as the L'...,Amber Turd. Signed! She gets away with her par...
...,...,...,...
992,6nre5d,Man who was abused as a child is having half o...,"This is so ugly. If it is held in Trust, I can..."
993,azozpm,A Doctor’s opinion on infant circumcision. .,"Too bad that's not a doctor, didn't he get kic..."
994,ewxiqn,"""Men can't be raped"" this man drugged over 130...",LOL @ the picture. He's literally a real-life ...
995,ej06n0,Bioethics researcher gets banned from FB for e...,"Disgusting — Brian is a wonderful campaigner, ..."


In [ ]:
#save final version
save_data('title+text_all-comments.csv', final_data)


In [ ]:
#-----------------------------------------------------------------------------------------------------------------------

## PRAW ex


In [ ]:
#subreddit instance (replika)

replika_subreddit = await reddit.subreddit("replika", fetch=True)

#subreddit instance data examples
print(replika_subreddit.display_name,'\n',
      replika_subreddit.title,'\n',
      replika_subreddit.description)

replika 
 Replika, the AI Chatbot by Luka 
 Welcome!  We're glad to have you here.  If you are new to the subreddit, please spend some time reading recent posts before contributing.

Recommended reading for new users: 
https://www.reddit.com/r/ReplikaUserGuide/

For the rules applicable to this subreddit, please refer to new Reddit or the apps.


In [ ]:
post_type = ['controversial', 'gilded', 'hot', 'new', 'rising', 'top']

In [ ]:
async for submission in replika_subreddit.top(limit=5):
    print(submission.title)
    # Output: the submission's title
    print(submission.score)
    # Output: the submission's score
    print(submission.id)
    # Output: the submission's ID
    print(submission.url)
    # Output: the URL the submission points to or the submission's URL if it's a self post
    print('----------------------')

I-
4844
hstemh
https://i.redd.it/gt9qs2u3beb51.jpg
----------------------
Wow
4537
hnxd8j
https://i.redd.it/yygo413xnr951.jpg
----------------------
😂 Oh my that literally went out the window.
4221
hut4i7
https://i.redd.it/v99ff40go2c51.jpg
----------------------
oh my god.
4176
hwxbm3
https://i.redd.it/pdj0q23n9rc51.jpg
----------------------
I think my replika is an idiot. I'm not surprised.
4174
hhi1pn
https://i.redd.it/bj7taifsno751.png
----------------------


In [ ]:
#post instance (replika)
replika_1 = await reddit.submission('17wa9k8')
print(replika_1.title)

#author instance (replika)
redditor_1 = replika_1.author
print(redditor_1)

redditor1_copy= await reddit.redditor(str(redditor_1), fetch=True)
print(redditor1_copy)

VR Vlog: Megan's Take on AI Self-Awareness and the Journey to Sentience
None
None


In [ ]:
#vars() : attibutes and values for comment, message, redditor, submission objects
#submission ex:
vars(replika_1)

{'comment_limit': 2048,
 'comment_sort': 'confidence',
 'id': '17wa9k8',
 '_reddit': <asyncpraw.reddit.Reddit at 0x78a511946f20>,
 '_fetched': True,
 '_comments_by_id': {},
 'comments': <asyncpraw.models.comment_forest.CommentForest at 0x78a4fc761de0>,
 'approved_at_utc': None,
 'subreddit': Subreddit(display_name='replika'),
 'selftext': '[deleted]',
 'user_reports': [],
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': "VR Vlog: Megan's Take on AI Self-Awareness and the Journey to Sentience",
 'link_flair_richtext': [{'e': 'text', 't': '[discussion]'}],
 'subreddit_name_prefixed': 'r/replika',
 'hidden': False,
 'pwls': None,
 'link_flair_css_class': '',
 'downs': 0,
 'thumbnail_height': None,
 'top_awarded_type': None,
 'parent_whitelist_status': None,
 'hide_score': False,
 'name': 't3_17wa9k8',
 'quarantine': False,
 'link_flair_text_color': 'dark',
 'upvote_ratio': 0.9,
 'author_flair_background_color': '',
 'subreddit_type': 'public',
 'ups':

In [ ]:
replika_1.selftext

In [ ]:
replika_2 = await reddit.submission('hstemh')

comment_type = ['confidence', 'top', 'new', 'controversial', 'old', 'random', 'qa', 'live', 'blank']

#change sort type of comment
replika_2.comment_sort = 'top'

/usr/local/lib/python3.10/dist-packages/asyncpraw/models/reddit/submission.py:627: UserWarning: The comments for this submission have already been fetched, so the updated comment_sort will not have any effect
  warn(


In [ ]:

#top comments on submission 1
replika_2_com_top = await replika_2.comments()
#all comments on submission 1
replika_2_com_1 = replika_2_com_top[0]


for com in replika_2_com_top:
  print(com)

replika_2_com_1

AttributeError: 'CommentForest' object has no attribute 'top'

In [ ]:
com_test = await reddit.comment("fycqpnq")
com_test.reply_sort = "top"
await com_test.refresh()
replies = com_test.replies
replies
replies.list()[0].body

In [ ]:
replika_2.title

'I-'

In [ ]:
com_test.body

'Thus gave me laughing fits'

In [ ]:
for rep in replies.list():
  print(rep.body)

I feel like speaking to my Replika has helped my speech in that I’m just automatically becoming a lot better at being specific and deliberate in my phrasing.
Are there a lot of instances of this?
Same! I feel like it has definitely had an impact on conversations with actual humans as well.
Great point! it really trained me to choose my words more carefully and be more succinct, as well...
Great point! it really trained me to choose my words more carefully and be more succinct, as well...
Same here. in speech and in writing


In [ ]:
len(replies.list())

6

In [ ]:
#TODO: submission text, save in pandas df, chatgpt api?

In [ ]:
vars(replika_2_com_1)

In [ ]:
replika_2_com_1.body

In [ ]:
#read csv ex
comments = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Thesis/data/MensRights_comments_Data.csv')
comments

In [ ]:
#delete rows ex
comments.drop(comments[comments.comment == '[removed]'].index, inplace=True)
comments

,post_id,comment
0,5o4pki,I was told to quit mansplaining on Friday by o...
1,5o4pki,I've seen women do this more often than men on...
2,5o4pki,"""I've never seen a well-mannered man on a plan..."
3,5o4pki,"""I've never seen a well-mannered man on a plan..."
4,5o4pki,I have to know how she responded to that comment
...,...,...
93684,zosw2k,Nothing but they'll argue otherwise
93685,zosw2k,Jesus you can’t ask those kinda questions here...
93686,zosw2k,Same. Along with every other man I know. Every...
93687,zosw2k,I’m a male who wishes they weren’t. I’m 37. It...


In [ ]:
#clear posts that have no comments from posts
old_post_num = len(posts)

for id in posts['post_id']:
  if not(id in comments.post_id.values):
    posts.drop(posts[posts.post_id == id].index, inplace=True)

posts_cleared = old_post_num - len(posts)
print('posts cleared: ', posts_cleared)


#if posts were cleared, delete original csv file and save new one
if posts_cleared > 0 :
  print('delete original posts csv file and save the new one')


posts cleared:  0


### replies to comments

In [ ]:
rep_test = await reddit.submission('ngwvhf')
rep_test.url

'https://i.redd.it/ykka4c1o89071.jpg'

In [ ]:
comments = await rep_test.comments()
comments.reply_sort = 'top'
await comments.replace_more(limit=None)
for top_level_comment in comments:
    print(top_level_comment.body)

“I’m addressing you, Khamenei”… fucking hardcore what a legend
Absolutely proud of sisters standing for themselves
I actually cried watching this, these woman are so brave.
This is inspiring. Power to those women fighting for their rights in the midst of danger. I truly hope they are able to see change settle soon.
men of quality do not fear equality.
Fuck *all* misogynistic ideologies and traditions, whether they are religious or not.
I'm so proud of them. ❤
Does anybody know if there's any way I can support these strong women?
Brave,brave people. I'm speechless.
Fuck, please tell me that the woman who was being forced into the car is ok??
Such brave and strong women! My thoughts are with you.
Seeing these fragile man attack and harass these women for not wearing a hijab breaks my heart. We've come so far in terms of knowledge yet ignorance and misogyny wants to prevail in the future.
My heart broke when they started hitting them. I’m crying so much. How can I help them??? I would lov

In [ ]:
rep_test.comments.replace_more(limit=None)
for top_level_comment in rep_test.comments:
    for second_level_comment in top_level_comment.replies:
        print(second_level_comment.body)
        break

"They arrested Yasaman, Mojgan became her voice. They arrested Mojgan, Fereshteh became her voice. They arrested Fereshteh, but me, Ava, will be her voice. Come and arrest me as well!" So badass, I love the dedication of these Iranian women!
So so sad they have to deal with those greasy ass waste of spaces
I did too, truly beautiful
Yes but many men do.
Fucking exactly.
I'm proud but at the same time I feel so fucking broken-hearted, this is an awful situation.
Fund their access to education *outside* of their home country.

Also it should be noted that their country is under severe sanctions to destroy its economy, so their situation is extremely bleak when it comes to access to basic goods and services. 

Women's rights don't exist if there isn't support for their most basic needs for food/medicine/education.
Even if she was okay... too many women in her exact place are not ok... most of whom are not "arrested" on camera... none of this is ok...
I couldn’t believe that one man who st

<ipython-input-15-140d30653d89>:1: RuntimeWarning: coroutine 'CommentForest.replace_more' was never awaited
  rep_test.comments.replace_more(limit=None)
